# This Notebook contains the code to filter, clean and manage the SRM raw data output.

### The following actions are carried out, please see each section for a more thorough explanation of what occurs

#### 0. Import necessary Python modules
#### 1. Import Raw Data and Standing data
#### 2. Filter out unwanted columns
#### 3. Clean dirty columns
#### 4. Add new columns that will be used for analysis later on.
#### 5. Generate output dataframe and export output

### 0. Import necesary Python Modules
For this project we will manipulate our data using the python library 'Pandas', which requires the use of the library 'NumPy'. These two libraries are imported as 'pd' and 'np' respectively, in line with convention. Additionally we use the datetime module, for superior handling of the dates in our data, and the workdays functions, which contains functions for calculating the differences between two dates without including weekends or holidays (similar to NETWORKDAYS in excel. For further information see here:

1. https://numpy.org/doc/stable/
2. https://pandas.pydata.org/docs/
3. https://docs.python.org/3/library/datetime.html
4. https://pypi.org/project/workdays/

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import workdays

### 1. Import Raw Data and Standing data
We import the raw data from a file which lies OUTSIDE of the repository, as it is too large to be contained on GitHub. As such, please will each user copy the pathname to their raw data.
1. LPG pathname: /Users/leogallagher/Documents/FMA-Work/KRG/PFM/Code/Expenditure/raw-data/srm-raw-data

There are 5 standing data dataframes in total, 3 of which are necessary to process the raw data. These are:

1. standing-data-au-ministry.csv - This associates each accounting unit with a Ministry
2. standing-data-current-status.csv - this data associates each of the current status' in the raw data with a status category which is useful to us
3. standing-data-ministry-analyst.csv - this identifies which analyst works on which ministry.

Each of these must be periodically checked to ensure that the are upto date and are accessible at GitHub at the following address:<br>
https://github.com/Leo-Gallagher/SRM-datapacks/tree/main/standing-data

If encountering issues with the import of the standing data make sure that the pathname exactly matches that on your local device, and that the file name is exactly correct also.

Copy the local pathname root for the standing data here for convenience
1. LPG pathname: /Users/leogallagher/Documents/FMA-Work/KRG/PFM/Code/Expenditure/srm-datapack/standing-data/

There is also an analysis date that must be UPDATED with the date that the current analysis is being completed to.

In [2]:
# define the root_path
root_path = r'/Users/leogallagher/Documents/FMA-Work/KRG/PFM/Code/Expenditure'
# Import Raw Data as df
df = pd.read_csv(root_path + r'/raw-data/srm-raw-data/230301-srm-raw-data.csv', low_memory=False)

#Import Standing data - ***YOU WILL NEED TO USE YOUR OWN PATH NAME***
# AU-Ministry Data
au_ministry_data = pd.read_csv(root_path + r'/srm-datapack/standing-data/standing-data-au-ministry.csv')

# Current Status data
current_status = pd.read_csv(root_path + r'/srm-datapack/standing-data/standing-data-current-status.csv')

# Ministry/Analyst Data
ministry_analyst_data = pd.read_csv(root_path + r'/srm-datapack/standing-data/standing-data-ministry-analyst.csv')

# KRG Holiday Data
krg_holidays = pd.read_csv(root_path + r'/srm-datapack/standing-data/standing-data-krg-holidays.csv')
# we will also need a list of these dates in the datetime format:
holiday_dates = pd.to_datetime(krg_holidays['holiday_date'])

In [3]:
# analysis date - *** To be updated each time this analysis is done. The format is (YYYY, (M)M, (D)D)
analysis_date = datetime(2023, 3, 1)

### 2. Filter out unwanted columns

Here we filter out most of the colums from the raw data as they are not necessary. The list provided in the code can be amended to include or remove columns.

In [4]:
# Columns to keep immediately, filtering out the many columns that are unneeded in the raw data
df_columns_to_keep = [
        'Srid','Date Submitted To Mofe','Approval Number Issued At','Ministry Input Id Issued At',
        'Ministry Id Issued At','Ministry Diwan Reviewer Action Date','Ministry Diwan Decision Maker Action Date',
        'Entity','Account Code','Exchange Rate','Total Cost Of Line','Amended Total Cost Of Line',
        'Savings Of Line','Total Sr Savings','Spending Team Action','Spending Team Action Date',
        'Price Evaluation Action','Price Evaluation Action Date','Engineering Directorate Action',
        'Engineering Directorate Action Date','Publications Directorate Action',
        'Publications Directorate Action Date','Director Of Spending Action','Director Of Spending Action Date',
        'Dg Pa Action','Dg Pa Action Date','Minister Of Finance Action','Minister Of Finance Action Date',
        'Com Action','Com Action Date', 'Second Round Ministry Diwan Reviewer Action Date',
        'Second Round Ministry Diwan Decision Maker Action Date','Second Round Spending Team Action Date',
        'Second Round Director Of Spending Action Date','Second Round Dg Pa Action Date',
        'Second Round Minister Of Finance Action Date','Second Round Com Action Date',
        'Second Round Final Approval Date','Final Approval Date','Current State']
df_filtered = df.loc[:, df_columns_to_keep]

### 3. Clean dirty columns (*** CHECK THIS SECTION IS CORRECT***)
Many of the columns have messy data in. A list of the cleaning processes is provided here:
1. 'Current State' Column - some of the entries are succeeded by a string of the form ' $XXXX', where the X's are placeholders for numbers. Theses elements of the string are removed.
2. Various Date Columns - these are cleaned to convert the date into a datetime format and to make sure that missing dates are appropriately marked.<br> 
    • Most missing dates are tagged with a '-' <br>
    • Some are simply missing and are marked as NaN.

3. Various Financial Columns: Data in some of the financial columns is missing, either as NaN or '-'. These are replaced (*** Check this is correct***). this applies to the following columns:<br>
    • 'Savings Of Line'
    • 'Amended Total Cost Of Line'<br>
    • 'Total Cost Of Line'>
    • 'Exchange Rate'<br>
After replacing missing data these are all converted to floats from strings so that we can perform numerical calculations.


In [5]:
# Current State Column: Remove characters preceded by a dollar sign from the strings in the Current State column.
df_filtered['Current State'] = df_filtered['Current State'].apply(lambda string: string.split(' $')[0])



# Date Columns: Format as datetime and deal with any missing values
# First define a list of the columns which include dates
date_cols = [
        'Date Submitted To Mofe','Approval Number Issued At','Ministry Input Id Issued At',
        'Ministry Id Issued At','Ministry Diwan Reviewer Action Date',
        'Ministry Diwan Decision Maker Action Date','Spending Team Action Date',
        'Price Evaluation Action Date','Engineering Directorate Action Date',
        'Publications Directorate Action Date','Director Of Spending Action Date',
        'Dg Pa Action Date', 'Minister Of Finance Action Date','Com Action Date',
        'Second Round Ministry Diwan Reviewer Action Date',
        'Second Round Ministry Diwan Decision Maker Action Date',
        'Second Round Spending Team Action Date',
        'Second Round Director Of Spending Action Date',
        'Second Round Dg Pa Action Date',
        'Second Round Minister Of Finance Action Date',
        'Second Round Com Action Date',
        'Second Round Final Approval Date','Final Approval Date']

# Loop through the relevant columns converting to datetime. This method also converts not date formats to the
# NaT (Not a Time) python generic time NA value. NOTE THAT THIS LOSES THE DIFFERENTATION BETWEEN ABSENT TIMES
# THAT WERE '-' AND WERE ORGINALLY NaN. 
for col in date_cols:
    df_filtered[col] = pd.to_datetime(df_filtered[col], 
                                      format='%Y-%m-%d %I:%M %p',
                                      errors='coerce').fillna(pd.NaT)
    

    
# Financial Columns: First define the columns to process
fin_cols = ['Savings Of Line','Amended Total Cost Of Line','Total Cost Of Line','Exchange Rate']

# Next replace '-' and NaN with 0, and convert to float for each column specified.
for col in fin_cols:
    df_filtered[col] = df_filtered[col].replace('-', 0).fillna(0).astype(float)

### 3. Add New Columns 
These new columns will be crucial for the creation of graphs and other data analysis. The following columns will be added.
1. 'Most Recent Action' - Calculated as the most recent date from all the date columns
2. 'User Currently With' - Found by merging raw data with the standing data 'current-status' 
3. 'Status' - Found by merging raw data with the standing data 'current-status' 
4. 'Status (Updated)' - Found by merging raw data with the standing data 'current-status' 
5. 'Final Status' - Found by checking a nubmer of columns (including 'Status' above) to determine the final status.
6. 'Line Item Savings IQD' - found as a multiple of the 'savings of line' column and the exchange rate
7. 'Amended Line Item IQD' - found as a multiple of the 'Amended Total Cost Of Line' column and the exchange rate
8. 'Original Line Item IQD' - found as a multiple of the 'Total Cost Of Line' column and the exchange rate
9. 'Original Total Cost of SR' - Found as the total cost for each SR, mapped back onto each line
10. 'Ministry' - Found by merging the Ministry/AU standing data onto the current data's AU column
11. 'Analyst' - Found by merging the Ministry/Analyst data on the newly created ministry column
12. 'days_since_last_action' - The difference in days between 'most recent action' and the analysis date
13. 'tot_days_processing' - The difference in days between the most recent action and the date first submitted.
14. 'analyst_days_processing' - the difference in days between the submission date and the analyst submitted date

In [6]:
### 'MOST RECENT ACTION' column

#columns to include are all the columns from which the 'most recent action' could be. I.e. the most recent action
# is the date that is most recent of the actions listed below.
columns_to_include = [
        'Date Submitted To Mofe','Approval Number Issued At','Ministry Input Id Issued At',
        'Ministry Id Issued At','Ministry Diwan Reviewer Action Date','Ministry Diwan Decision Maker Action Date',
        'Spending Team Action Date',
        'Price Evaluation Action Date',
        'Engineering Directorate Action Date','Publications Directorate Action Date',
        'Director Of Spending Action Date','Dg Pa Action Date',
        'Minister Of Finance Action Date','Com Action Date',
        'Second Round Ministry Diwan Reviewer Action Date','Second Round Ministry Diwan Decision Maker Action Date',
        'Second Round Spending Team Action Date','Second Round Director Of Spending Action Date',
        'Second Round Dg Pa Action Date','Second Round Minister Of Finance Action Date',
        'Second Round Com Action Date','Second Round Final Approval Date','Final Approval Date']

# Determine the max of the included columns and write in as a new column to df_filtered
df_filtered['Most Recent Action'] = df_filtered[columns_to_include].max(axis=1)

In [7]:
### 'User Currently With', 'Status', 'Status [Updated]' Columns

# First ensure that all characters in the 'current state' columns of each dataframe are all uppercase, to ensure
# matches when merging
current_status['Current State'] = current_status['Current State'].str.upper()
df_filtered['Current State'] = df_filtered['Current State'].str.upper()

# Merge standing data 'standing-data-current-status and df_filtered to get the status columns
df_filtered = pd.merge(df_filtered, current_status,how='left', on='Current State')

In [8]:
### 'FINAL STATUS' column

# First we define the function that determines what the status of each row.
def final_status(row):
    if row['Current State'] == "REVOKED" and row['Date Submitted To Mofe'] == pd.NaT:
        return "Not Submitted"
    
    elif row['Current State'] == "REVOKED" and row['Final Approval Date'] != pd.NaT:
        if row['Minister Of Finance Action'] == "Approved By Minister Of Finance" or row['Minister Of Finance Action'] == "Amended and Approved By Minister Of Finance" or  row['Minister Of Finance Action'] == "Pending Tender":
            return "Revoked After Approval"
        
        elif row['Com Action'] == "Approved By Council Of Ministers" or row['Com Action'] == "Amended And Approved By Council Of Ministers" or row['Com Action'] == "Pending Tender By Council Of Ministers":
            return "Revoked After Approval"
        
        elif row['Minister Of Finance Action'] == "Rejected By Minister Of Finance" or row['Com Action'] == "Rejected By Council Of Ministers":

            return "Revoked After Rejection"
        
    elif row['Status'] == 'In process':
        return 'In Process'
    
    elif row['Status'] == 'Not submitted':
        return 'Not Submitted'
    
    elif row['Status'] == 'Revoked':
        return 'Revoked'
    
    elif row['Status'] == 'Rejected':
        return 'Rejected'
    
    elif row['Status'] == 'Approved':
        if row['Total Sr Savings'] == 0:
            return 'Approved'
        else:
            return 'Amended'
    else:
        return np.nan

# Once the function is defined we apply it row-wise to df_filtered
df_filtered['Final Status'] = df_filtered.apply(final_status, axis=1)

In [9]:
### 'Line Item Savings IQD', 'Amended Line Item IQD', 'Original Line Item IQD' columns

# These columns can all be calculated as one line functions, as they are equal to df_filtered columns
# multiplied by the equivalent entry in the 'exchange rate' column.

### ADD 'LINE ITEM SAVINGS IQD'
df_filtered['Line Item Savings IQD'] = df_filtered.apply(lambda row:
                                                         row['Exchange Rate'] * row['Savings Of Line'],
                                                         axis=1)
### ADD 'Amended Line Item IQD'
df_filtered['Amended Line Item IQD'] = df_filtered.apply(lambda row:
                                                         row['Exchange Rate'] * row['Amended Total Cost Of Line'],
                                                         axis=1)
### Add 'Original Line Item IQD'
df_filtered['Original Line Item IQD'] = df_filtered.apply(lambda row:
                                                          row['Exchange Rate'] * row['Total Cost Of Line'],
                                                          axis=1)

In [10]:
### Original Total Cost of SR' Column

# Group the rows of the DataFrame by the values in 'Srid',
# and calculate the sum of 'Original Line Item IQD' for each group
sum_original = df_filtered.groupby('Srid')['Original Line Item IQD'].sum()

# Create a new column 'Original Total Cost of SR' with the calculated values, mapped onto the relevant SRIDs
df_filtered['Original Total Cost of SR'] = df_filtered['Srid'].map(sum_original)

In [11]:
### 'Ministry'

## Formed by merging au/ministry standing data with df_filtered on 'Entity' column (which contains AU names)
df_filtered = pd.merge(df_filtered, au_ministry_data,how='left', on='Entity')

In [12]:
### 'Analyst'

### Formed by merging au/ministry data with df_filtered on 'ministry' column
df_filtered = pd.merge(df_filtered, ministry_analyst_data,how='left', on='Ministry')

In [13]:
# Add 'days_since_last_action' column, calculated as the difference between the analysis date
# and the most recent action
df_filtered['days_since_last_action'] = df_filtered.apply(lambda row:
                                                          (analysis_date - row['Most Recent Action']).days,
                                                          axis=1)

In [14]:
### Add the 'tot_days_processing' column

# Define a function to calculate the total processing time. This uses the 'workdays' package's 'networkdays'
# and the KRG holidays standing data
def tot_processing_time(row):
    if pd.isna(row['Date Submitted To Mofe']) or pd.isna(row['Most Recent Action']):
        days_processing = -1
    elif row['Final Status'] == 'In Process':
        days_processing = workdays.networkdays(row['Date Submitted To Mofe'], analysis_date,
                                               holiday_dates)
    else:
        days_processing = workdays.networkdays(row['Date Submitted To Mofe'],row['Most Recent Action'],
                                              holiday_dates)
    return days_processing

# Apply function rows-wise
df_filtered['tot_days_processing'] = df_filtered.apply(tot_processing_time, axis=1)

In [15]:
### Add the 'analyst_days_processing' column

# Define a function to calculate the analyst processing time
def analyst_processing_time(row):
    if pd.isna(row['Date Submitted To Mofe']) or pd.isna(row['Spending Team Action Date']):
        days_processing = 0
    elif row['Final Status'] == 'In Process':
        days_processing = workdays.networkdays(row['Date Submitted To Mofe'], analysis_date,
                                              holiday_dates)
    else:
        days_processing = workdays.networkdays(row['Date Submitted To Mofe'],row['Spending Team Action Date'],
                                              holiday_dates)
    return days_processing

# Apply function rows-wise
df_filtered['analyst_days_processing'] = df_filtered.apply(analyst_processing_time, axis=1)

### 5. Generate output dataframe and export output

We filter out the remaining columns which are not necessary for the production of graphs and charts (they were necessary before to create the additional columns). Finally we export the dataframe as a csv file so that it can be used in google sheets.

In [16]:
## Define the columns we wish to keep
output_cols = ['Srid', 'Date Submitted To Mofe', 'Account Code', 'Most Recent Action',
       'User Currently With', 'Final Status','Line Item Savings IQD', 'Amended Line Item IQD',
       'Original Line Item IQD', 'Original Total Cost of SR', 'Ministry',
       'Analyst', 'days_since_last_action', 'tot_days_processing',
       'analyst_days_processing']

# Filter the dataframe on these columns and write to a new dataframe, df_final
df_final = df_filtered[output_cols]

In [17]:
## You will need to write in your own path to find the data in your own machine.
df_final.to_csv(root_path + r'/processed-data/srm-processed-data/' + 
                analysis_date.strftime("%y%m%d") +r'-srm-data.csv',
                index=False)

In [18]:
df_final.head()

,Srid,Date Submitted To Mofe,Account Code,Most Recent Action,User Currently With,Final Status,Line Item Savings IQD,Amended Line Item IQD,Original Line Item IQD,Original Total Cost of SR,Ministry,Analyst,days_since_last_action,tot_days_processing,analyst_days_processing
0,SRID-1,2020-07-20 22:29:00,2-01-02-09-02-00,2020-09-20 13:26:00,Ministry - Rejected,Rejected,0.0,341000000.0,341000000.0,806000000.0,MOTAC,Helen,891.0,41,2
1,SRID-1,2020-07-20 22:29:00,2-01-04-07-00-00,2020-09-20 13:26:00,Ministry - Rejected,Rejected,0.0,62000000.0,62000000.0,806000000.0,MOTAC,Helen,891.0,41,2
2,SRID-1,2020-07-20 22:29:00,2-01-03-08-02-00,2020-09-20 13:26:00,Ministry - Rejected,Rejected,0.0,155000000.0,155000000.0,806000000.0,MOTAC,Helen,891.0,41,2
3,SRID-1,2020-07-20 22:29:00,2-01-03-08-02-00,2020-09-20 13:26:00,Ministry - Rejected,Rejected,0.0,31000000.0,31000000.0,806000000.0,MOTAC,Helen,891.0,41,2
4,SRID-1,2020-07-20 22:29:00,2-01-03-08-02-00,2020-09-20 13:26:00,Ministry - Rejected,Rejected,0.0,217000000.0,217000000.0,806000000.0,MOTAC,Helen,891.0,41,2
